In [7]:
import pandas as pd
from bs4 import BeautifulSoup as BSoup
from bs4 import Comment
from session import Session 
import csv
from helper import Helper

In [8]:
session = Session(1000000) # doesn't really matter
helper = Helper()

In [9]:
url = "https://statiz.sporki.com/team/" # starting point
front_url = "https://statiz.sporki.com" # front part of URL in which hrefs will be concatenated on.

In [13]:
# to filter duplicate
# remove the header row
added_player = set([tuple(row) for row in csv.reader(open('statiz_name_map.csv', 'r'))][1:])

In [14]:
print(len(added_player))

614


In [4]:
team_data = session.fetch(url).content.decode("utf-8") # each team's data

In [5]:
trimmed = BSoup(team_data, "lxml").find(class_="team_list")

In [6]:
with open('statiz_name_map.csv', 'wa', newline='') as file:

    writer = csv.writer(file)
    
    header = ["name_kor", "name_eng"]
    writer.writerow(header)

    teams = trimmed.find_all(class_="select_con")
    
    for team in teams:
        years = team.find_all("a")
    
        for year in years:
            current_ct_code, current_year = helper.team_url_to_code_and_year(year['href'])
            team_player_list_url = helper.team_url_to_back_number_url(year['href'])
            team_data = session.fetch(team_player_list_url).content.decode("utf-8")
            team_html = BSoup(team_data, "lxml")
            players = team_html.find_all(class_="item away")[2:]

            for player in players:
                player_url = front_url + player.find("a")['href']
                player_data = session.fetch(player_url).content.decode("utf-8")
                player_html = BSoup(player_data, "lxml")
                player_name_proto = player_html.find(class_="name")
                # sometimes there is a blank page that doesn't contain any info about the player.
                if not player_name_proto:
                    continue
                player_name = player_name_proto.text.strip()
                player_info = (player_name.split('(')[0].strip(), player_name.split('(')[1][:-1])
    
                if player_info not in added_player:
                    added_player.add(player_info)
                    writer.writerow(player_info)
                    print("newly added: ", player_info)
                else:
                    print("dup skipped: ", player_info)

            print(f"finished code:{current_ct_code}, year: {current_year}")

newly added:  ('김일권', 'Il-Kwon Kim')
newly added:  ('김경훈', 'Kyung-Hoon Kim')
newly added:  ('조충열', 'Chung-Yeol Cho')
newly added:  ('차영화', 'Young-Hwa Cha')
newly added:  ('김종모', 'Jong-Mo Kim')
newly added:  ('김종윤', 'Jong-Yoon Kim')
newly added:  ('김준환', 'Jun-Hwan Kim')
newly added:  ('김성한', 'Sung-Han Kim')
newly added:  ('홍순만', 'Soon-Man Hong')
newly added:  ('이상윤', 'Sang-Yoon Lee')
newly added:  ('강만식', 'Man-Sik Kang')
newly added:  ('임정면', 'Jung-Myun Lim')
newly added:  ('김우근', 'Woo-Geun Kim')
newly added:  ('방수원', 'Soo-Won Bang')
newly added:  ('박전섭', 'Jeon-Sub Park')
newly added:  ('최영조', 'Young-Jo Choi')
newly added:  ('신태중', 'Tae-Joong Shin')
newly added:  ('김봉연', 'Bong-Yeon Kim')
newly added:  ('김용만', 'Yong-Man Kim')
newly added:  ('김용남', 'Yong-Nam Kim')
finished code:2, year: 1982
dup skipped:  ('조충열', 'Chung-Yeol Cho')
dup skipped:  ('김일권', 'Il-Kwon Kim')
newly added:  ('김일환', 'Il-Hwan Kim')
dup skipped:  ('차영화', 'Young-Hwa Cha')
newly added:  ('서정환', 'Jung-Hwan Seo')
dup skip

AttributeError: 'NoneType' object has no attribute 'text'